<a href="https://colab.research.google.com/github/reyerick88-ai/openai-chatkit-starter-app/blob/main/IA_CALL_CENTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip -q install openai-whisper ffmpeg-python pandas
!apt-get -qq update
!apt-get -qq install -y ffmpeg


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [38]:
from google.colab import files
uploaded = files.upload()
audio_path = next(iter(uploaded.keys()))
audio_path


Saving audio_prueba_llamada_es_con_silencios.wav to audio_prueba_llamada_es_con_silencios (3).wav


'audio_prueba_llamada_es_con_silencios (3).wav'

In [39]:
import whisper, json

model = whisper.load_model("small")  # "base" es más rápido; "small" suele ser mejor en ES
result = model.transcribe(audio_path, language="es", word_timestamps=False)

# Guardar transcripción completa
with open("transcript.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("Listo. Segmentos:", len(result["segments"]))
print(result["text"][:300])


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Listo. Segmentos: 9
 Hola, gracias por llamar a soporte. ¿En qué te puedo ayudar? Hola, tengo un cargo que no reconozco en mi tarjeta. Entiendo. Voy a validar la información. ¿Qué te haces? ¡Listo! Veo un cargo en línea del día 10. ¿Tú lo hiciste? No, yo no. De acuerdo. Voy a levantar el reporte y bloquear la tarjeta. 


In [40]:
import pandas as pd

segments = result["segments"]

silences = []
for i in range(1, len(segments)):
    prev_end = segments[i-1]["end"]
    cur_start = segments[i]["start"]
    gap = cur_start - prev_end
    if gap > 0:
        silences.append({
            "from_end_s": prev_end,
            "to_start_s": cur_start,
            "silence_s": gap
        })

df = pd.DataFrame(silences)

# Umbrales (ajústalos)
LONG_SILENCE = 3.0   # silencios "largos" > 3s
VERY_LONG = 6.0      # muy largos > 6s

total_silence = df["silence_s"].sum() if len(df) else 0
count_long = (df["silence_s"] > LONG_SILENCE).sum() if len(df) else 0
count_very_long = (df["silence_s"] > VERY_LONG).sum() if len(df) else 0

print("Silencio total (s):", round(total_silence, 2))
print("Silencios > 3s:", int(count_long))
print("Silencios > 6s:", int(count_very_long))

# Guardar CSV
df.to_csv("silences.csv", index=False)
df.head(10)


Silencio total (s): 15.72
Silencios > 3s: 2
Silencios > 6s: 0


,from_end_s,to_start_s,silence_s
0,4.64,8.52,3.880000e+00
1,12.52,15.36,2.840000e+00
2,20.52,24.52,4.000000e+00
3,29.52,31.52,2.000000e+00
4,33.52,34.52,1.000000e+00
5,41.52,41.52,7.105427e-15
6,44.52,46.52,2.000000e+00


In [41]:
call_minutes = segments[-1]["end"]/60 if segments else 0

report = []
report.append("REPORTE BÁSICO DE LLAMADA")
report.append(f"Duración aprox (min): {call_minutes:.2f}")
report.append(f"Silencio total (s): {total_silence:.2f}")
report.append(f"Silencios > {LONG_SILENCE:.0f}s: {int(count_long)}")
report.append(f"Silencios > {VERY_LONG:.0f}s: {int(count_very_long)}")
report.append("")
report.append("TOP 10 silencios más largos (segundos):")

if len(df):
    top = df.sort_values("silence_s", ascending=False).head(10)
    for _, r in top.iterrows():
        report.append(f"- {r['silence_s']:.2f}s (fin {r['from_end_s']:.2f}s → inicio {r['to_start_s']:.2f}s)")
else:
    report.append("- No se detectaron silencios entre segmentos.")

report_text = "\n".join(report)
print(report_text)

with open("reporte.txt", "w", encoding="utf-8") as f:
    f.write(report_text)


REPORTE BÁSICO DE LLAMADA
Duración aprox (min): 0.84
Silencio total (s): 15.72
Silencios > 3s: 2
Silencios > 6s: 0

TOP 10 silencios más largos (segundos):
- 4.00s (fin 20.52s → inicio 24.52s)
- 3.88s (fin 4.64s → inicio 8.52s)
- 2.84s (fin 12.52s → inicio 15.36s)
- 2.00s (fin 29.52s → inicio 31.52s)
- 2.00s (fin 44.52s → inicio 46.52s)
- 1.00s (fin 33.52s → inicio 34.52s)
- 0.00s (fin 41.52s → inicio 41.52s)


In [42]:
import json

call_seconds = segments[-1]["end"] if segments else 0

metrics = {
    "call_duration_s": float(call_seconds),
    "call_duration_min": float(call_seconds/60 if call_seconds else 0),
    "total_silence_s": float(total_silence),
    "total_silence_min": float(total_silence/60),
    "silence_pct": float((total_silence/call_seconds*100) if call_seconds else 0),
    "silences_gt_3s": int((df["silence_s"] > 3.0).sum()) if len(df) else 0,
    "silences_gt_6s": int((df["silence_s"] > 6.0).sum()) if len(df) else 0,
    "top_silences": df.sort_values("silence_s", ascending=False).head(10).to_dict("records") if len(df) else []
}

with open("metrics.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

print("Listo: metrics.json generado")


Listo: metrics.json generado


In [43]:
!pip -q install requests
import requests

WEBHOOK_URL = "https://hook.us2.make.com/pw8peir9ydknxjy7pi30cseypuq6n4kw"  # tu webhook

payload = {
    "metrics": metrics,
    "transcript_text": result["text"][:6000]
}

resp = requests.post(WEBHOOK_URL, json=payload, timeout=30)
print("Make status:", resp.status_code)
print("Make respuesta:", resp.text[:200])


Make status: 200
Make respuesta: Accepted


In [44]:
from google.colab import files
files.download("metrics.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
from google.colab import files
files.download("reporte.txt")
files.download("silences.csv")
files.download("transcript.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>